In [1]:
from sympy.physics.quantum import Commutator, Operator
import sympy
from sympy import *
from IPython.display import display
import numpy as np

# Hamiltonian

Define operators and constants

In [2]:
U, F, delta_a, delta_b, J, kappa_a,eta = symbols(r'U F \Delta_a \Delta_b J \kappa_a \eta')

a = Symbol(r'a', commutative=False)
ad = Symbol(r'a^\dag', commutative=False)
b = Symbol(r'b', commutative=False)
bd = Symbol(r'b^\dag', commutative=False)

rho = Symbol(r'\rho', commutative=False)

Construct Hamiltonian

In [3]:
Ham1 = J*(ad*b+a*bd) + F*(ad+a) -delta_a*ad*a -delta_b*bd*b  + Rational(1, 2)*U*ad*ad*a*a
Ham2 = -delta_a*ad*a -delta_b*bd*b
Ham3 = Mul(U,ad,ad,a,a,evaluate=False)
Ham4 = UnevaluatedExpr(Mul(Rational(1, 2)*U,ad,ad,a,a,evaluate=False))


MyHam = 0
display(MyHam)

0

Jump operators

In [4]:

def getJump(constant,c,rho):
    cd = Dagger(c)
    return UnevaluatedExpr(constant*(cd*rho*c - Rational(1, 2)*(c*cd*rho + rho*c*cd)))

D1 = kappa_a*(ad*rho*a - Rational(1, 2)*(a*ad*rho + rho*a*ad))
D2 = eta*(a*a*rho*ad*ad - Rational(1, 2)*(ad*ad*a*a*rho + rho*ad*ad*a*a))
#D1 = getJump(kappa_a,a,rho) !!! Not working because the Dagger() is not recognized as one of the defined symbols when applying the operators!
#display(D1)

    

# Lindblad master equation

In [5]:
ME_comm = -sympy.I* (Commutator(MyHam, rho)) + D2

display(ME_comm)
ME= ME_comm.doit()
#display(ME)
ME = expand(ME)
display(ME)

\eta*(a**2*\rho*a^\dag**2 - (\rho*a^\dag**2*a**2 + a^\dag**2*a**2*\rho)/2)

-\eta*\rho*a^\dag**2*a**2/2 + \eta*a**2*\rho*a^\dag**2 - \eta*a^\dag**2*a**2*\rho/2

# Convert to desired phase space function

In [6]:
from phasespaceconversion import pow_to_mul_sep2,PhaseSpaceFunction

In [7]:
clean_term = ad*rho*a
display(clean_term)
clean_termNoRho =  clean_term.subs(rho,1 )
print(clean_term.args)

def getPosRho(clean_term,rho):
    counter=0
    items= clean_term.args
    n_items = len(items)
    print(items)
    for k in range(n_items):
        item = items[k]
        if item==rho:
           break    # break here
        else:
            [op, n] = item.as_base_exp()
            counter = counter+n
            print(counter)

    print('rho occurs on position  {}'.format(counter))
    return counter

def labelOperators(ordered_operators, pos_rho):
    n_operators = len(ordered_operators)
    operator_labels = ['RHS']*n_operators
    operator_labels[:pos_rho]=['LHS']*pos_rho
    return operator_labels

pos_rho=getPosRho(clean_term,rho)
ordered_operators = pow_to_mul_sep2(clean_termNoRho)

operator_labels=labelOperators(ordered_operators, pos_rho)

print(ordered_operators)
print(operator_labels)
operator_labels =[] # label each individual operator in the list as RHS or LHS of Rho
print(clean_termNoRho.args)

a^\dag*\rho*a

(a^\dag, \rho, a)
(a^\dag, \rho, a)
1
rho occurs on position  1
[a^\dag, a]
['LHS', 'RHS']
(a^\dag, a)


In [8]:
# ordered_operators = pow_to_mul_sep2(clean_termNoRho)
# display(ordered_operators)



Wigner

In [9]:
myWigner = PhaseSpaceFunction(ME,'W')
W_eqn = myWigner.getFPfromME()
print('Result')
display(expand(W_eqn))

(\eta*a**2*\rho*a^\dag**2,
 -\eta*\rho*a^\dag**2*a**2/2,
 -\eta*a^\dag**2*a**2*\rho/2)

\eta*a**2*\rho*a^\dag**2

a**2*\rho*a^\dag**2

a**2*a^\dag**2

(a**2, \rho, a^\dag**2)
1
rho occurs on position  1
j=
0
Term 0 :
ops left in mid term, order operators applied:
[a, a]


...

a^\dag**2*a**2

Term 2 :
LHS term, order operators applied:
[a, a, a^\dag, a^\dag]


-\eta*(\alpha*(\alpha*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2)) + 2*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha) + Derivative(W(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2), \alpha^*)/2)/4 + \alpha*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha)/2 + \alpha*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2), \alpha^*)/8 - \alpha^**(\alpha*(\alpha*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha) + W(\alpha, \alpha^*, \beta, \beta^*) + Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*)/2) + \alpha*W(\alpha, \alpha^*, \beta, \beta^*) + \alpha*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*)/2 + Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, (\alpha^*, 2))/4)/2 + \alpha^**(-\alpha*(\alpha*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha) + W(\alpha, \alpha^*, \beta, \beta^*) + Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha

Result


\alpha*\eta*Abs(\alpha**2)*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha) + \alpha*\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha) + \alpha*\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2), \alpha^*)/4 + \alpha^**\eta*Abs(\alpha**2)*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + \alpha^**\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + \alpha^**\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, (\alpha^*, 2))/4 + 4*\eta*W(\alpha, \alpha^*, \beta, \beta^*)*Abs(\alpha**2) + 2*\eta*Abs(\alpha**2)*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*)

HusimiQ

In [10]:
myQ= PhaseSpaceFunction(ME,'Q')
Q_eqn = myQ.getFPfromME()
print('Result')
display(expand(Q_eqn))

(\eta*a**2*\rho*a^\dag**2,
 -\eta*\rho*a^\dag**2*a**2/2,
 -\eta*a^\dag**2*a**2*\rho/2)

\eta*a**2*\rho*a^\dag**2

a**2*\rho*a^\dag**2

a**2*a^\dag**2

(a**2, \rho, a^\dag**2)
1
rho occurs on position  1
j=
0
Term 0 :
ops left in mid term, order operators applied:
[a, a]


...

a^\dag**2*a**2

Term 2 :
LHS term, order operators applied:
[a, a, a^\dag, a^\dag]


-\alpha^***2*\eta*(\alpha*(\alpha*Q(\alpha, \alpha^*, \beta, \beta^*) + Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)) + \alpha*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), (\alpha^*, 2)))/2

Result


\alpha**2*\eta*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2))/2 + \alpha*\eta*Abs(\alpha**2)*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha) + 4*\alpha*\eta*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha) + 2*\alpha*\eta*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2), \alpha^*) + \alpha^***2*\eta*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), (\alpha^*, 2))/2 + \alpha^**\eta*Abs(\alpha**2)*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + 4*\alpha^**\eta*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + 2*\alpha^**\eta*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha, (\alpha^*, 2)) + 4*\eta*Q(\alpha, \alpha^*, \beta, \beta^*)*Abs(\alpha**2) + 2*\eta*Q(\alpha, \alpha^*, \beta, \beta^*) + 4*\eta*Abs(\alpha**2)*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*) + 4*\eta*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*) + \eta*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2), (\

P representation

In [11]:
myP= PhaseSpaceFunction(ME,'P')
P_eqn = myP.getFPfromME()
print('Result')
display(expand(P_eqn))

(\eta*a**2*\rho*a^\dag**2,
 -\eta*\rho*a^\dag**2*a**2/2,
 -\eta*a^\dag**2*a**2*\rho/2)

\eta*a**2*\rho*a^\dag**2

a**2*\rho*a^\dag**2

a**2*a^\dag**2

(a**2, \rho, a^\dag**2)
1
rho occurs on position  1
j=
0
Term 0 :
ops left in mid term, order operators applied:
[a, a]


...

a^\dag**2*a**2

Term 2 :
LHS term, order operators applied:
[a, a, a^\dag, a^\dag]


-\eta*(-\alpha**2*\alpha^**Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha) + \alpha**2*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2)) - 2*\alpha*\alpha^**P(\alpha, \alpha^*, \beta, \beta^*) + 4*\alpha*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha) + \alpha^**(\alpha**2*\alpha^**P(\alpha, \alpha^*, \beta, \beta^*) - \alpha**2*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha) - 2*\alpha*P(\alpha, \alpha^*, \beta, \beta^*)) + 2*P(\alpha, \alpha^*, \beta, \beta^*))/2

Result


-\alpha**2*\eta*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2))/2 + \alpha*\eta*Abs(\alpha**2)*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha) - 2*\alpha*\eta*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha) - \alpha^***2*\eta*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), (\alpha^*, 2))/2 + \alpha^**\eta*Abs(\alpha**2)*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) - 2*\alpha^**\eta*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + 4*\eta*P(\alpha, \alpha^*, \beta, \beta^*)*Abs(\alpha**2) - 2*\eta*P(\alpha, \alpha^*, \beta, \beta^*)

# Separate Real and imaginary Parts Wigner

## Not working for now, use mathematica file instead

In [12]:
W_exp_comp = expand(W_eqn)
display(W_exp_comp)

# redefine symbols in original equation (not so nice that this has to be repeated but okay)

#field = Function('W')(alpha,self.alphas,self.beta,self.betas) # can be W, Q or P
alpha = Symbol(r'\alpha', commutative=True)
alphas = Symbol(r'\alpha^*', commutative=True)
beta = Symbol(r'\beta', commutative=True)
betas = Symbol(r'\beta^*', commutative=True)

# define real and imag parts:
alpha_r = Symbol(r'a_R', commutative=True)
alpha_i = Symbol(r'a_I', commutative=True)
beta_r = Symbol(r'b_R', commutative=True)
beta_i = Symbol(r'b_I', commutative=True)

# substitute in expression
W_exp_comp_sep =W_exp_comp.subs(alpha, alpha_r+sympy.I*alpha_i )
W_exp_comp_sep =W_exp_comp.subs(alphas, alpha_r-sympy.I*alpha_i )
W_exp_comp_sep = W_exp_comp.subs(beta, beta_r+sympy.I*beta_i )
W_exp_comp_sep = W_exp_comp.subs(betas, beta_r-sympy.I*beta_i )

display(W_exp_comp_sep)
W_exp_Re = re(W_exp_comp)
display(W_exp_Re)

\alpha*\eta*Abs(\alpha**2)*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha) + \alpha*\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha) + \alpha*\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2), \alpha^*)/4 + \alpha^**\eta*Abs(\alpha**2)*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + \alpha^**\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + \alpha^**\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, (\alpha^*, 2))/4 + 4*\eta*W(\alpha, \alpha^*, \beta, \beta^*)*Abs(\alpha**2) + 2*\eta*Abs(\alpha**2)*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*)

\alpha*\eta*Abs(\alpha**2)*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha) + \alpha*\eta*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha) + \alpha*\eta*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), (\alpha, 2), \alpha^*)/4 + \alpha^**\eta*Abs(\alpha**2)*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha^*) + \alpha^**\eta*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha^*) + \alpha^**\eta*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha, (\alpha^*, 2))/4 + 4*\eta*W(\alpha, \alpha^*, \beta, -I*b_I + b_R)*Abs(\alpha**2) + 2*\eta*Abs(\alpha**2)*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha, \alpha^*)

4*re(\eta*W(\alpha, \alpha^*, \beta, \beta^*))*Abs(\alpha**2) + 2*re(\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, \alpha^*))*Abs(\alpha**2) + re(\alpha*\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha))*Abs(\alpha**2) + re(\alpha*\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha)) + re(\alpha*\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), (\alpha, 2), \alpha^*))/4 + re(\alpha^**\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*))*Abs(\alpha**2) + re(\alpha^**\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)) + re(\alpha^**\eta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha, (\alpha^*, 2)))/4

In [13]:
W_exp_comp = expand(W_eqn)
W_exp_comp.atoms()
W_exp_comp.atoms(Function)

{Abs(\alpha**2), W(\alpha, \alpha^*, \beta, \beta^*)}

In [14]:
latex(W_exp_comp)

'\\alpha \\eta \\left|{\\alpha^{2}}\\right| \\frac{\\partial}{\\partial \\alpha} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)} + \\alpha \\eta \\frac{\\partial}{\\partial \\alpha} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)} + \\frac{\\alpha \\eta \\frac{\\partial^{3}}{\\partial \\alpha^{*}\\partial \\alpha^{2}} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)}}{4} + \\alpha^{*} \\eta \\left|{\\alpha^{2}}\\right| \\frac{\\partial}{\\partial \\alpha^{*}} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)} + \\alpha^{*} \\eta \\frac{\\partial}{\\partial \\alpha^{*}} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)} + \\frac{\\alpha^{*} \\eta \\frac{\\partial^{3}}{\\partial \\left(\\alpha^{*}\\right)^{2}\\partial \\alpha} W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)}}{4} + 4 \\eta W{\\left(\\alpha,\\alpha^{*},\\beta,\\beta^{*} \\right)} \\left|{\\alpha^{2}}\\right| + 2 \\eta \\left|{\\alpha^{2}}\\right| \\frac{\\partial^{2}}{\\partial \